# Getting a global entry interview appointment with browser automation

## Background
I've filled out the paperwork to get [global entry](https://www.cbp.gov/travel/trusted-traveler-programs/global-entry), paid $100, and gotten conditionally approved. All that's left is the interview. It turns out that there are zero interview slots available for the next two years. Yes, really, I called to check. Also, walk-ins are not allowed. This seems to be a [known issue](https://www.sfchronicle.com/business/networth/article/Want-to-schedule-a-Global-Entry-interview-at-SFO-14835643.php).

One solution is to do [enrollment on arrival](https://www.cbp.gov/travel/trusted-traveler-programs/global-entry/enrollment-arrival) next time you're returning to the US from an international trip. Appointments also sporadically appear (because of cancellations?), and you can try to snap one up. There's a [twitter bot](https://twitter.com/GOESbot) which watches for and announces these sporadic slots. From the activity of the bot, I saw there were several openings per day, but they were always gone by the time I got to them. So I wrote this tool to check for appointments every minute and notify me if it finds one.

## Yeah yeah yeah ... how do I use this?

* Get conditionally approved for global entry.
* `pip install selenium`
* Download a [webdriver binary](https://selenium-python.readthedocs.io/installation.html#drivers) (here I assume ChromeDriver 80).
* Put this ipynb in the same directory as the `chromedriver` binary. If you use some other driver, change the cell that starts the webdriver below.
* Create a file called `login.py` in the same directory with the following contents:
  ```
  EMAIL = 'your_email@email.com'
  PASSWORD = 'W2rbVeFE3]kXwLu}'
  ```
* If you want an appointment someplace other than SFO, change `center_id` below. You can find the center id by logging in and inspecting the page source on the appointment page.
* Run this notebook. It'll open a browser window, login to your account (you'll have to use your security key or OTP code for the first login), navigate to the appointment page, and click the link to fetch the next available appointment every minute. When it finds an available appointment, it'll announce it through your speakers.
* When an available appointment is found, simply finish booking it through the webdriver browser window.

## Disclaimers

This document is for educational/entertainment purposes only. It is not intended to be executed by a computing machine. Moreover, if it were hypothetically executed by a computing machine, there may well be bugs, especially if there have been any changes to the trusted traveler program web site since this short story was authored. The author does not assume any liability for mental anguish you may experience in contemplating these errors.

In [ ]:
import login  # Contains email and password ... you must create this file yourself.

center_id = 'US34'  # SFO
#center_id = 'US33'  # San Diego
seconds_between_tries = 60
max_tries = 1 * 60 * 60 // seconds_between_tries

In [ ]:
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import subprocess

# Start webdriver

In [ ]:
driver = webdriver.Chrome('./chromedriver')

# Login

The first time you login after starting the webdriver, you'll need to use a security key or OTP code.

In [ ]:
def log_in():
    driver.get("https://ttp.cbp.dhs.gov")
    time.sleep(1)
    driver.find_element_by_css_selector('input.login-button').click()
    time.sleep(.5)
    driver.find_element_by_css_selector('button.btn-primary').click()
    time.sleep(.5)
    driver.find_element_by_id('user_email').send_keys(login.EMAIL)
    driver.find_element_by_id('user_password').send_keys(login.PASSWORD)
    driver.find_element_by_css_selector('input.btn-primary').click()
    time.sleep(10)
    
log_in()

Go to appointment scheduling page

In [ ]:
def go_to_page():
    b = driver.find_element_by_css_selector('div.middle')
    b.find_element_by_css_selector('button.btn-primary').click()
    time.sleep(3)
    
go_to_page()

# Try to find an appointment

In [ ]:
def wait_till_appointments_fetched(max_tries=100):
    while max_tries > 0:
        try:
            s = driver.find_element_by_css_selector('div.spinner-mask')
            time.sleep(.2)
            max_tries -= 1
        except NoSuchElementException:
            break
            
def notify(text):
    print(text)
    subprocess.call(['spd-say', text])
    subprocess.call(['notify-send', '--urgency=normal', text])

for try_num in range(max_tries):
    try:
        driver.find_element_by_id(f'centerDetails{center_id}').click()
        wait_till_appointments_fetched()
        box = driver.find_element_by_id(f'popover{center_id}')
        appt_text = box.find_element_by_css_selector('div.nextAppointment').text
        if appt_text == 'No appointments available for this location':
            if try_num % 10 == 0:
                print(appt_text)
        else:
            notify(appt_text)
            break
        time.sleep(seconds_between_tries)
        driver.find_element_by_id(f'popover{center_id}BtnClosePopover').click()
    except ElementClickInterceptedException:
        # We were logged out.
        log_in()
        go_to_page()
else:
    notify('Finished trying without success')

# Shutdown webdriver

In [ ]:
driver.close()